# Moving alginment workflow

I have been refactoring the aln workflow. Now I want to move the output over to its new location. 

In [1]:
# %load ../start.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-10-26 
Git hash: 6748af63c6a424a3619f31f0f44a74e5901a3070


In [2]:
import shutil
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [3]:
store = pd.HDFStore('../../sra.h5', mode='r')

In [4]:
def move(fname, srx=None, srr=None):
    orig = os.path.join('../../output/alignment/raw/{srx}/{srr}', fname).format(srx=srx, srr=srr)
    new = os.path.join('../../aln-wf/output/samples/{srx}/{srr}', fname).format(srx=srx, srr=srr)
    os.makedirs(os.path.dirname(new), exist_ok=True)
    
    try:
        shutil.move(orig, new)
    except FileNotFoundError as e:
        pass

In [5]:
patterns = [
    '{srr}_1.trim.clean.fastq.gz.log',
    '{srr}.fq.bam.bamtools.stats',
    '{srr}.fq.bam.counts',
    '{srr}.fq.bam.counts.log',
    '{srr}.fq.bam.counts.jcounts',
    '{srr}.fq.bam.counts.summary',
    '{srr}.fq.bam.log',
    '{srr}.fq.bam.samtools.idxstats',
    '{srr}.fq.bam.samtools.stats',
    '{srr}.fq.bam.tsv',
    'ALIGNMENT_BAD'
]

In [6]:
dfs = []
for i, record in store['ids'].iterrows():
    if os.path.exists('../../output/alignment/raw/{srx}/{srr}'.format(**record.to_dict())):
        for p in patterns:
            dfs.append(delayed(move)(p, **record.to_dict()))

In [7]:
len(dfs)

192126

In [8]:
with ProgressBar():
    compute(*dfs)

[########################################] | 100% Completed | 27min 41.6s


In [9]:
def move(fname, srx=None):
    orig = os.path.join('../../output/alignment/raw/{srx}', fname).format(srx=srx)
    new = os.path.join('../../aln-wf/output/samples/{srx}', fname).format(srx=srx)
    try:
        shutil.move(orig, new)
    except FileNotFoundError as e:
        pass

In [10]:
patterns = [
    '{srx}.counts'
]

In [11]:
srxs = set(store['ids'].srx.tolist())

In [12]:
dfs = []
for srx in srxs:
    if os.path.exists('../../output/alignment/raw/{srx}'.format(srx=srx)):
        for p in patterns:
            dfs.append(delayed(move)(p, srx=srx))

In [13]:
print(len(dfs))

11601


In [14]:
with ProgressBar():
    compute(*dfs)

[########################################] | 100% Completed |  2min 38.5s
